Download GNPS data <br>
Replace Job ID xy below with your GNPS job ID:

In [1]:
#! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=xy&view=download_cytoscape_data' -o ../GNPS_output/Network.zip
#! unzip -d ../GNPS_output/Network ../GNPS_output/Network.zip

specify your NAP job ID:

In [2]:
nap_id1 = 'xy'
nap_id2 = 'xy'

load libraries

In [3]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = '/Applications/MarvinSuite/bin/' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [5]:
if 'clusterinfo_summary' in os.listdir('../GNPS_output/Network/') and 'DB_result' in os.listdir('../GNPS_output/Network/'):
    netfile = '../GNPS_output/Network/clusterinfo_summary/' + str(os.listdir('../GNPS_output/Network/clusterinfo_summary/')[0]) 
    gnpslibfile = '../GNPS_output/Network/DB_result/'+ str(os.listdir('../GNPS_output/Network/DB_result/')[0]) 
elif 'clusterinfosummarygroup_attributes_withIDs_withcomponentID' in os.listdir('../GNPS_output/Network/'):
    netfile = '../GNPS_output/Network/clusterinfosummarygroup_attributes_withIDs_withcomponentID/' + str(os.listdir('../GNPS_output/Network/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[0])
    gnpslibfile = '../GNPS_output/Network/result_specnets_DB/'+ str(os.listdir('../GNPS_output/Network/result_specnets_DB/')[0])
else:
    netfile = '../GNPS_output/Network/clusterinfosummary/' + str(os.listdir('../GNPS_output/Network/clusterinfosummary/')[0])
    gnpslibfile = '../GNPS_output/Network/result_specnets_DB/'+ str(os.listdir('../GNPS_output/Network/result_specnets_DB/')[0]) 

In [6]:
#derep = pd.read_csv('Derep_output/'+[s for s in os.listdir('Derep_output') if "DEREPLICATOR" in s][0], sep = '\t')

In [7]:
nap1 = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id1+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
nap2 = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id2+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t',error_bad_lines=False)

In [8]:
gnpslib.shape

(122, 46)

add all chemical structural information output as dataframe items in list

In [9]:
matches = [gnpslib, nap1, nap2]

In [10]:
out = unique_smiles(matches)

In [13]:
out['df'].to_csv("../output/SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [17]:
! molconvert inchikey:SAbs ../output/SMILES.csv{csv:strucSMILES} -o ../output/InchiKeys.txt

In [18]:
ikeys = pd.read_csv("../output/InchiKeys.txt",  sep='\t',header = None)

out['df']["inchikey"] = ikeys
inchi_dic = make_inchidic(out)

ikeys.columns = ["InChIKey"]
ikeys.to_csv("../output/InchiKeys.txt", quoting=csv.QUOTE_NONE, escapechar='&')

retrieve ClassyFire classifcations

In [19]:
get_classifications("../output/InchiKeys.txt")

1000
2000
3000
4000


In [20]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [21]:
final.head()

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
0,1,15,33,Organic compounds,0.997576,Organoheterocyclic compounds,0.244969,Carboxylic acids and derivatives,0.158717,NaN,0.183604,Coumarins and derivatives,0.0330884,Aromatic heteropolycyclic compounds,0.403326
1,71,15,33,Organic compounds,0.997576,Organoheterocyclic compounds,0.244969,Carboxylic acids and derivatives,0.158717,NaN,0.183604,Coumarins and derivatives,0.0330884,Aromatic heteropolycyclic compounds,0.403326
2,82,15,33,Organic compounds,0.997576,Organoheterocyclic compounds,0.244969,Carboxylic acids and derivatives,0.158717,NaN,0.183604,Coumarins and derivatives,0.0330884,Aromatic heteropolycyclic compounds,0.403326
3,115,15,33,Organic compounds,0.997576,Organoheterocyclic compounds,0.244969,Carboxylic acids and derivatives,0.158717,NaN,0.183604,Coumarins and derivatives,0.0330884,Aromatic heteropolycyclic compounds,0.403326
4,125,15,33,Organic compounds,0.997576,Organoheterocyclic compounds,0.244969,Carboxylic acids and derivatives,0.158717,NaN,0.183604,Coumarins and derivatives,0.0330884,Aromatic heteropolycyclic compounds,0.403326


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [22]:
final.to_csv("../output/ClassyFireResults_Network.txt", sep = '\t', index = False)

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [23]:
len(set(list(ikeys.InChIKey)) - set(list(df.inchikey)))/len(set(list(ikeys.InChIKey)))

0.005963302752293578

which InChIKeys could not be classified?

In [24]:
set(list(ikeys.InChIKey)) - set(list(df.inchikey))

{'InChIKey=AIFUEUDNKHXATH-LDLOPFEMSA-N',
 'InChIKey=BDLOYQXKGAKUOW-KXBFYZLASA-O',
 'InChIKey=BSLFBXWCUALIAR-CYBMUJFWSA-M',
 'InChIKey=CCSZJBCZZCTGRR-XRXFFOJZSA-M',
 'InChIKey=GROJVGYPODRKQN-UHFFFAOYSA-M',
 'InChIKey=GXBRKDCCCPVLQF-LDLOPFEMSA-N',
 'InChIKey=IEKOTSCYBBDIJC-LJQANCHMSA-M',
 'InChIKey=JAHDAIPFBPPQHQ-UHFFFAOYSA-N',
 'InChIKey=JRLWFBQZMUBEJU-KLMICZSBSA-N',
 'InChIKey=KZYURKFVECTBBF-DIPNUNPCSA-N',
 'InChIKey=LDWLOOALTYTPDA-FVTFTPSCSA-M',
 'InChIKey=LJFXMYJWGWDOHP-OPNZPCBVSA-M',
 'InChIKey=OYDSYYITLGQRHW-UHFFFAOYSA-M',
 'InChIKey=PBZMYKRPZYCWSZ-KLMICZSBSA-N',
 'InChIKey=PLXXRMIOJWENQI-OAHLLOKOSA-M',
 'InChIKey=QLVHBWUZDKPUCA-LDLOPFEMSA-N',
 'InChIKey=QSMWBKZZGXMYCO-AWEZNQCLSA-M',
 'InChIKey=SDOFJRBZPHZENA-ZDUSSCGKSA-M',
 'InChIKey=SEFBYRGDYPSSFP-KDXMTYKHSA-N',
 'InChIKey=TYESGYJYLAMSIN-LDLOPFEMSA-N',
 'InChIKey=VQFBHHBERXAYKU-HDJSIYSDSA-P',
 'InChIKey=WEXRBKRFCBLWEL-UHFFFAOYSA-N',
 'InChIKey=WSNOMGCXCMXPNC-ATVHPVEESA-N',
 'InChIKey=XDSSPSLGNGIIHP-GSVOUGTGSA-M',
 'InChIKey=YGTOB

create graphML file (optional)

In [25]:
#if any("FEATURE" in s for s in os.listdir('GNPS_output_graphML/')):
    #graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'FEATURE' in x][0]
    #graphML = read_graphml(graphMLfile)
    #graphML_classy = make_classyfire_graphml(graphML,final)
    #nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
#elif any("METABOLOMICS" in s for s in os.listdir('GNPS_output_graphML/')):
    #graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'METABOLOMICS' in x][0]
    #graphML = read_graphml(graphMLfile)
    #graphML_classy = make_classyfire_graphml(graphML,final)
    #nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
#else:
    #print('There is no graphML file for this GNPS molecular network job')

In [4]:
import session_info


session_info.show()